# Resim Sınıflandırma ve Veri Manipülasyonu
Bu projede, 10 farklı sınıfı içeren bir görüntü veri seti üzerinden, görüntü sınıflandırma modeli oluşturulacak. Ayrıca, veri manipülasyon teknikleri kullanarak modelin doğruluğu üzerinde farklı ışık koşulları, renk sabitliği gibi etkilerle performans testleri yapılacaktır.

### Proje Adımları:
1. **Veri Setini Yükleme:** Veri setinden resimler yüklenecek ve etiketlerle birlikte sınıflandırılacak.
2. **Veri Artırma:** Eğitim verisi üzerinde veri artırma teknikleri (augmentation) uygulanarak modelin genelleme yeteneği artırılacak.
3. **Modelin Tasarlanması:** Derin öğrenme tabanlı bir CNN modeli oluşturulacak.
4. **Modelin Eğitilmesi:** Eğitim verisi kullanılarak model eğitilecek ve doğruluk ölçülecek.
5. **Veri Manipülasyonu:** Veri üzerinde ışık manipülasyonu ve renk sabitliği teknikleri uygulanacak.
6. **Sonuçların Karşılaştırılması:** Manipüle edilmiş ve orijinal verilerle yapılan testlerin sonuçları karşılaştırılacak.


## Gerekli Kütüphanelerin Yüklenmesi
Bu adımda, projemizin ihtiyaç duyduğu kütüphaneleri import edeceğiz. Bu kütüphaneler, veri işleme, model oluşturma, görselleştirme ve görüntü manipülasyonu işlemleri için gereklidir.

- **os:** Dosya yollarını yönetmek için
- **numpy:** Sayısal işlemler için
- **PIL (Pillow):** Görüntü işleme için
- **sklearn.model_selection:** Veri setini eğitim ve test olarak ayırmak için
- **cv2 (OpenCV):** Görüntü manipülasyonu (parlaklık artırma vb.) için
- **matplotlib:** Görselleştirme işlemleri için
- **tensorflow.keras:** Derin öğrenme modeli oluşturmak ve eğitmek için


In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


## Veri Setinin Yüklenmesi
Bu adımda, resimlerimizi ve etiketlerini yükleyeceğiz. Resimler, `JPEGImages` klasöründe yer alıyor ve her biri 10 farklı sınıfa ait. Sınıflar şunlardır:

- Collie
- Dolphin
- Elephant
- Fox
- Moose
- Rabbit
- Sheep
- Squirrel
- Giant Panda
- Polar Bear

Verileri yüklerken, her resmin boyutunu 128x128 piksel olarak ayarlayacağız ve değerlerini normalize edeceğiz (0-1 arasında olacak şekilde). Ayrıca, her sınıf için etiketleri de belirleyeceğiz.


In [ ]:
# Resimlerin bulunduğu ana dizini belirlemek
image_dir = "C:\\Users\\omers\\Desktop\\dataset\\JPEGImages"

# 10 sınıfın adlarını girmek
classes = ['collie', 'dolphin', 'elephant', 'fox', 'moose', 'rabbit', 'sheep', 'squirrel', 'giant panda', 'polar bear']

# Resimleri ve etiketlerini yüklemek
def load_images(image_dir, classes, image_size=(128, 128)):
    images = []
    labels = []
    
    for label, class_name in enumerate(classes):
        class_dir = os.path.join(image_dir, class_name)
        for image_name in os.listdir(class_dir)[:650]:  # Her sınıftan 650 resim
            image_path = os.path.join(class_dir, image_name)
            image = Image.open(image_path)
            image = image.resize(image_size)  # Resimleri aynı boyutta yap
            image = np.array(image) / 255.0  # Resimleri normalize et
            images.append(image)
            labels.append(label)  # Etiketleri ekle
    
    return np.array(images), np.array(labels)

# Veriyi yüklemek
X, y = load_images(image_dir, classes)


## Eğitim ve Test Setlerine Ayırma
Veri setini eğitim ve test setlerine ayıracağız. Test seti için verimizin %30'unu ayırarak, kalan %70'i ile modelimizi eğiteceğiz. Bu şekilde modelin doğruluğunu gerçek dünya verisiyle değerlendirebileceğiz.


In [ ]:
# Eğitim ve test setlerini ayırmak
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Eğitim ve test boyutlarını yazdırmak
print(f"Eğitim seti boyutu: {X_train.shape}")
print(f"Test seti boyutu: {X_test.shape}")


## Veri Artırma (Augmentation)
Veri artırma, modelin daha çeşitli ve genel sonuçlar vermesini sağlamak için kullanılır. Eğitim setinde çeşitli manipülasyonlar (dönme, kaydırma, zoom) uygulayarak modelin daha fazla örnekle eğitilmesini sağlarız. Bu sayede modelin overfitting (aşırı öğrenme) yapma ihtimali azalır.

Uygulanan veri artırma işlemleri:
- **Dönme (rotation)**: Resimleri rastgele döndürme
- **Kaydırma (shift)**: Resimleri yatay veya dikey kaydırma
- **Zoom**: Resimleri rastgele yakınlaştırma
- **Yatay çevirme (horizontal flip)**: Resimleri tersine çevirme


In [ ]:
# Eğitim verisi için veri artırma (augmentation) kullanmak
datagen = ImageDataGenerator(
    rotation_range=30,    # Resimleri döndürme
    width_shift_range=0.2,  # Yatay kaydırma
    height_shift_range=0.2, # Dikey kaydırma
    shear_range=0.2,      # Kesme dönüşümü
    zoom_range=0.2,       # Zoom
    horizontal_flip=True, # Yatay çevirme
    fill_mode='nearest'   # Boş bölgeleri doldurma
)

# Eğitim verisi için augmentation uygulamak
datagen.fit(X_train)


## CNN Modelinin Tasarlanması
Bu adımda, bir Konvolüsyonel Sinir Ağı (CNN) modeli oluşturacağız. CNN'ler, görüntü verileriyle çalışırken oldukça başarılıdır çünkü görüntülerdeki yerel özellikleri öğrenebilirler. Modelimizde şu katmanlar bulunacaktır:
- **Konvolüsyonel katmanlar**: Görüntüden özellik çıkarımı yapacak
- **MaxPooling katmanları**: Özellik haritalarını küçülterek hesaplama maliyetini azaltacak
- **Fully Connected (Dense) katmanlar**: Sınıflandırma işlemini yapacak
- **Dropout katmanı**: Overfitting'i önlemek için rastgele nöronları kapatacak


In [ ]:
# Modeli oluşturmak
model = models.Sequential()

# Konvolüsyonel katmanlar
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Fully connected (dense) katman
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))  # Dropout ile overfitting'i engelle
model.add(layers.Dense(len(classes), activation='softmax'))  # Son katman, 10 sınıf

# Modeli derle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


## Modelin Eğitilmesi
Bu aşamada, modelimizi eğitim verisi ile 10 epoch boyunca eğiteceğiz. Her epoch sonunda, modelin doğruluğu test verisi üzerinde kontrol edilecek. Eğitim sırasında veri artırma işlemi uygulanarak modelin genelleme gücü artırılacaktır.


In [ ]:
# Modeli eğit
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))

# Modeli test et
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test seti doğruluğu: {test_acc * 100:.2f}%")


## Resimlerin Farklı Işık Koşullarında Manipüle Edilmesi
Veri artırma dışında, test setini manipüle ederek modelin farklı ışık koşullarında nasıl performans gösterdiğini test edeceğiz. Burada, parlaklık seviyelerini artıracağız.

In [ ]:
# Resimleri farklı ışık koşullarında manipüle etme
def get_manipulated_images(image):
    # Resmi farklı ışık koşulları altında manipüle etme
    image_brightness = cv2.convertScaleAbs(image, alpha=1.2, beta=50)  # Parlaklık artırma
    return image_brightness


## Manipüle Edilmiş Test Seti ile Modeli Denemek
Manipüle edilmiş test verisi ile modelimizi test ediyoruz. Bu, modelimizin ışık koşullarına karşı dayanıklılığını ölçmemize yardımcı olacak.

In [ ]:
# Manipüle edilmiş test seti ile modeli test et
X_test_manipulated = np.array([get_manipulated_images(img) for img in X_test])
test_loss, test_acc = model.evaluate(X_test_manipulated, y_test)
print(f"Manipüle edilmiş test seti doğruluğu: {test_acc * 100:.2f}%")


## Renk Sabitliği Uygulama (Gray World Algoritması)
Modelimizin renk dengesizliği konusunda daha dayanıklı olması için Gray World algoritmasını uygulayacağız. Bu algoritma, resimlerdeki renkleri sabitleyerek farklı ışık koşullarında daha tutarlı sonuçlar elde etmemizi sağlar.

In [ ]:
# Gray World algoritmasını kullanarak renk sabitliği
def get_wb_images(image):
    avg_r = np.mean(image[:, :, 0])
    avg_g = np.mean(image[:, :, 1])
    avg_b = np.mean(image[:, :, 2])
    
    avg = (avg_r + avg_g + avg_b) / 3
    image[:, :, 0] = image[:, :, 0] * (avg / avg_r)  # Red channel
    image[:, :, 1] = image[:, :, 1] * (avg / avg_g)  # Green channel
    image[:, :, 2] = image[:, :, 2] * (avg / avg_b)  # Blue channel
    
    return np.clip(image, 0, 255).astype(np.uint8)


##  Renk Sabitliği Uygulanmış Test Seti ile Denemek
Son olarak, renk sabitliği uygulanmış test verileri ile modelimizi test edeceğiz. Bu işlem, renk dengesizlikleri karşısında modelin performansını ölçmek için önemlidir.

In [ ]:
# Renk sabitliği uygulanmış test seti ile testi yap
X_test_wb = np.array([get_wb_images(img) for img in X_test])
test_loss, test_acc = model.evaluate(X_test_wb, y_test)
print(f"Renk sabitliği uygulanmış test seti doğruluğu: {test_acc * 100:.2f}%")


## Sonuçları Karşılaştırma
Son olarak, modelimizin eğitim ve test doğruluklarını karşılaştırarak genel başarısını değerlendireceğiz.

In [ ]:
# Sonuçları karşılaştırma
print(f"Eğitim seti doğruluğu: {history.history['accuracy'][-1]*100:.2f}%")
print(f"Test seti doğruluğu: {test_acc*100:.2f}%")
